# Tool Calling Use Cases

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#GET API KEYS
from utils import get_tavily_api_key
TAVILY_API_KEY = get_tavily_api_key()

In [ ]:
#Load helper functions

In [3]:
from utils import llama31
from utils import cprint #printing the python dictionaries in colorful format
import json

## Define tool system prompt

In [4]:
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime("%d %B %Y")
print(formatted_date)

18 October 2024


In [5]:
tool_system_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

## The brave_search built-in tool

In [6]:
#getting wwather results using tool call in brave search
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [7]:
response = llama31(prompt) #call to the brave search built in toll
print(response)



brave_search.call(query="Menlo Park California weather")


In [8]:
#don't need to call tool
no_tool_call_prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the population of California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [9]:
no_tool_call_response = llama31(no_tool_call_prompt)
print(no_tool_call_response)



As of the 2020 United States Census, the population of California is approximately 39.5 million people. However, please note that this information may have changed slightly since the last census, and the current population may be higher due to ongoing growth.


### Calling the search API

In [10]:
#seearch in web using tavily client

from tavily import TavilyClient #import tavily client
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

result = tavily_client.search("current weather in Menlo Park, California")
cprint(result)

{
    "query": "current weather in Menlo Park, California",
    "follow_up_questions": null,
    "answer": null,
    "images": [],
    "results": [
        {
            "title": "Weather in Menlo Park, California",
            "url": "https://www.weatherapi.com/",
            "content": "{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1729266450, 'localtime': '2024-10-18 08:47'}, 'current': {'last_updated_epoch': 1729266300, 'last_updated': '2024-10-18 08:45', 'temp_c': 13.2, 'temp_f': 55.8, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 4.5, 'wind_kph': 7.2, 'wind_degree': 327, 'wind_dir': 'NNW', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 35, 'cloud': 0, 'feelslike_c': 12.9, 'feelslike_f': 55.2, 'windchill_c': 10.5, 'win

### Reprompting Llama with search tool response

In [11]:
search_result = result["results"][0]["content"]
print(search_result)

{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1729266450, 'localtime': '2024-10-18 08:47'}, 'current': {'last_updated_epoch': 1729266300, 'last_updated': '2024-10-18 08:45', 'temp_c': 13.2, 'temp_f': 55.8, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 4.5, 'wind_kph': 7.2, 'wind_degree': 327, 'wind_dir': 'NNW', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 35, 'cloud': 0, 'feelslike_c': 12.9, 'feelslike_f': 55.2, 'windchill_c': 10.5, 'windchill_f': 50.9, 'heatindex_c': 11.9, 'heatindex_f': 53.5, 'dewpoint_c': -1.5, 'dewpoint_f': 29.2, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 7.2, 'gust_kph': 11.6}}


In [13]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>{response}<|eom_id|>
<|start_header_id|>ipython<|end_header_id|> 

{search_result}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
#added search result as a past conversation
#<|python_tag|> --> response of the tool search(brave search) is added using python tag
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 18 October 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>

brave_search.call(query="Menlo Park California weather")<|eom_id|>
<|start_header_id|>ipython<|end_header_id|> #added search result as a past conversation

{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1729266450, 'localtime': '2024-10-18 08:47'}, 'current': {'last_updated_epoch': 1729266300, 'last_updated': '2024-10-18 08:45', 'temp_c': 13.2, 'temp_f': 55.8, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 4.5, 'wi

In [14]:
response = llama31(prompt)
print(response)

The current weather in Menlo Park, California is sunny with a temperature of 55.8°F (13.2°C) and a wind speed of 7.2 mph (11.6 km/h) from the NNW. The humidity is 35% and the pressure is 30.08 in (1019.0 mb).


## Using the higher-level message

In [16]:
#define system prompt message use brave search tool
system_prompt_content = f"""
Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

In [17]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",   "content": "What is the current weather in Menlo Park, California?"}
  ]

In [18]:
response = llama31(messages)
print(response)

brave_search.call(query="Menlo Park California weather")


In [19]:
messages = [
    {"role": "system",     "content":  system_prompt_content},
    {"role": "user",       "content": "What is the current weather in Menlo Park, California?"},
    {"role": "assistant",  "content": response},
    {"role": "ipython",    "content": search_result} #getting search result
  ]

In [20]:
response = llama31(messages)
print(response)

The current weather in Menlo Park, California is sunny with a temperature of 13.2°C (55.8°F) and a wind speed of 4.5 mph (7.2 km/h) coming from the NNW. The humidity is 35% and the pressure is 1019.0 mb (30.08 in). There is no precipitation and the visibility is 10.0 km (6.0 miles). The UV index is 0.0.


The Wolfram Alpha tool

In [22]:
math_problem = "Can you help me solve this equation: x^3 - 2x^2 - x + 2 = 0?"

In [26]:
messages = [
    {"role" : "system", "content" : system_prompt_content},
    {"role" : "user", "content" : math_problem}
]

In [27]:
response = llama31(messages)
print(response)

wolfram_alpha.call(query="solve x^3 - 2x^2 - x + 2 = 0")


In [ ]:
#Calling the Wolframe Alpha tool for solve math problem

In [28]:
from utils import wolfram_alpha
tool_result = wolfram_alpha("solve x^3 - 2x^2 - x + 2 = 0")
print(tool_result)

x = ± 1
x = 2


### Checking the result using sympy

In [29]:
from sympy import symbols, Eq, solve

x = symbols('x')                           # Define the variable
equation = Eq(x**3 - 2*x**2 - 1*x + 2, 0) # Define the equation
solution = solve(equation, x)              # Solve the equation

print(solution)

[-1, 1, 2]


### Reprompting Llama with Wolfram Alpha tool response

In [30]:
messages = [
    {"role": "system", "content": system_prompt_content},
    {"role": "user",      "content": math_problem},
    #adding the result of tool call and reprompt
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": tool_result}
]

In [31]:
response = llama31(messages)
print(response)

The equation x^3 - 2x^2 - x + 2 = 0 has three solutions: x = 1, x = -1, and x = 2.


## The code interpreter built-in tool

In [37]:
loan_question = (
    "How much is the monthly payment, total payment, "
    "and total interest paid for a 30 year mortgage of $1M "
    "at a fixed rate of 6% with a 20% down payment?"
)

In [38]:
messages = [
    {"role": "system",     "content": system_prompt_content},
    {"role": "user",       "content": loan_question},
  ]

In [39]:
response = llama31(messages)
print(response)

wolfram_alpha.call("monthly payment, total payment, total interest paid for a 30 year mortgage of $800000 at 6% with a 20% down payment")


# Calculate loan payments

In [40]:
from utils import calculate_loan #use calculate_loan function from utils for calculation
monthly_payment, total_payment, total_interest_paid = calculate_loan(
    loan_amount = 1000000, 
    annual_interest_rate = 0.06, 
    loan_term = 30, 
    down_payment = 200000)

print(f"Monthly payment: ${(monthly_payment)}")
print(f"Total payment: ${(total_payment)}")
print(f"Total interest paid: ${(total_interest_paid)}")

Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705


## Generating the code in Java

In [41]:
messages = [
    {"role":    "system",     
     "content": system_prompt_content + "\nGenerate the code in Java."},
    {"role":    "user",       "content": loan_question},
  ]

In [42]:
#calculation code in java
response = llama31(messages)
print(response)

import java.util.Scanner;

public class MortgageCalculator {
    public static void main(String[] args) {
        // Input values
        double principal = 1000000; // $1M
        double downPayment = 0.2 * principal; // 20% down payment
        double interestRate = 0.06; // 6% interest rate
        int years = 30; // 30 years

        // Calculate the loan amount
        double loanAmount = principal - downPayment;

        // Calculate the monthly interest rate
        double monthlyInterestRate = interestRate / 12;

        // Calculate the number of payments
        int numberOfPayments = years * 12;

        // Calculate the monthly payment
        double monthlyPayment = loanAmount * monthlyInterestRate * Math.pow(1 + monthlyInterestRate, numberOfPayments) / (Math.pow(1 + monthlyInterestRate, numberOfPayments) - 1);

        // Calculate the total payment
        double totalPayment = monthlyPayment * numberOfPayments;

        // Calculate the total interest paid
        doubl

### Reprompting Llama with Code Interpreter tool response

In [43]:
code_interpreter_tool_response="""
Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705
"""

In [44]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",      "content": loan_question},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": code_interpreter_tool_response}#result of the excuting code
  ]

In [45]:
response = llama31(messages)
print(response)

The monthly payment for a 30-year mortgage of $1M at a fixed rate of 6% with a 20% down payment is $4,796. The total payment over the life of the loan is $1,726,705, and the total interest paid is $926,705.


## Llama 3.1 custom tool calling

In [46]:
from utils import trending_songs, get_boiling_point#see utils to get what are the trending songs and other quiries

country = "US"
top_num = 5
top_songs = trending_songs(country, top_num)
print(f"Top {top_num} trending songs in {country}:")
print(top_songs)

Top 5 trending songs in US:
['Blinding Lights - The Weeknd', 'Levitating - Dua Lipa', 'Peaches - Justin Bieber', 'Save Your Tears - The Weeknd', 'Good 4 U - Olivia Rodrigo']


## Prompt Llama 3.1 for custom tool call

In [47]:
#we use costom tool functions from utils

user_prompt = """
Answer the user's question by using the following functions if needed.
If none of the functions can be used, please say so.
Functions (in JSON format):
#1.get boiling point
{
    "type": "function", "function": {
        "name": "get_boiling_point",
        "description": "Get the boiling point of a liquid",
        "parameters": {
            "type": "object", "properties": [
                {"liquid_name": {"type": "object", "description": "name of the liquid"}},
                {"celsius": {"type": "object", "description": "whether to use celsius"}}
            ], "required": ["liquid_name"]
        }
    }
}
#2.trending song function 
{
    "type": "function", "function": {
        "name": "trending_songs",
        "description": "Returns the trending songs on a Music site",
        "parameters": {
            "type": "object", "properties": [
                {"country": {"type": "object", "description": "country to return trending songs for"}},
                {"n": {"type": "object", "description": "The number of songs to return"}}
            ], "required": ["country"]
        }
    }
}

Question: Can you check the top 5 trending songs in US?
"""

In [48]:
messages = [
    {
      "role": "system", "content":  f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""},
    {"role": "user", "content": user_prompt}
  ]

In [49]:
result = llama31(messages,405)
print(result)

{"type": "function", "name": "trending_songs", "parameters": {"country": "US", "n": "5"}}


### Calling the custom tool

In [50]:
#all are getting from utils
custom_tools = {"trending_songs": trending_songs,
                "get_boiling_point": get_boiling_point}

In [51]:
res = json.loads(result)
function_name = res['name']
parameters = list(res['parameters'].values())
function_name, parameters

('trending_songs', ['US', '5'])

In [52]:
tool_result = custom_tools[function_name](*parameters)
tool_result

['Blinding Lights - The Weeknd',
 'Levitating - Dua Lipa',
 'Peaches - Justin Bieber',
 'Save Your Tears - The Weeknd',
 'Good 4 U - Olivia Rodrigo']

### Reprompting Llama with custom tool call result

In [53]:
messages = [
    {
      "role": "system", "content":  f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""},
    {"role": "user", "content": user_prompt},
    {"role": "assistant", "content": result},
    {"role": "ipython", "content": ','.join(tool_result)}
  ]

In [54]:
response = llama31(messages, 70)
print(response)

The top 5 trending songs in the US are:

1. Blinding Lights - The Weeknd
2. Levitating - Dua Lipa
3. Peaches - Justin Bieber
4. Save Your Tears - The Weeknd
5. Good 4 U - Olivia Rodrigo
